In [1]:
from pathlib import Path
from typing import List

import pandas as pd
import numpy as np

YEAR = 2022


A line magic that can help to download the data and store it in variables
`input_path` and `data` to use directly in the code after:

It will use:
* if no argument: current day and year
* one argument: the day set as argument + current year
* two arguments: the day and the year

```python
%get_daily_input 2
data[:100]
```

In [2]:
!pip install -q advent-of-code-data

You should consider upgrading via the 'C:\AnsysDev\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
from pathlib import Path
from typing import cast
from datetime import datetime

from aocd import get_data
from aocd import submit

from IPython import get_ipython  # type: ignore
from IPython.core.interactiveshell import InteractiveShell  # type: ignore
from IPython.core.magic import register_line_cell_magic  # type: ignore
from IPython.display import HTML  # type: ignore


@register_line_cell_magic
def get_daily_input(line:str) -> None:
    """Retrieve the input for the current day and storing variables
    * input_path: Path to the input file
    * data: Content of the input file
    """
    ipython = cast(InteractiveShell, get_ipython())

    today = datetime.today()
    current_year = today.year
    current_day = today.day
    if line:
        args = [int(e) for e in line.split()] + [current_year]
    else:
        args = [current_day, current_year]
    day, year = args[:2]

    input_dir = Path("inputs")
    input_dir.mkdir(exist_ok=True)
    input_path = input_dir / f"{day:02d}.txt"
    input_path_html = f'<a href="{input_path.as_posix()}">{input_path.as_posix()}</a>'

    if not input_path.exists():
        ipython.system(f"aocd {day} {year} > {input_path}")
        ipython.run_cell(f"HTML('✅Input saved to {input_path_html}')")
    else:
        ipython.run_cell(f"HTML('✔️Input already exists in {input_path_html}')")

    ipython.run_cell(f"input_path = Path('{input_path.as_posix()}')")
    ipython.run_cell(f"data = get_data(day={day}, year={year})")
    msg = "✅Variables <code>input_path</code> and <code>data</code> set"
    ipython.run_cell(f"HTML('{msg}')")

# 01 December 2022


In [7]:
%get_daily_input 1 2022

In [8]:
def get_calories(path: Path) -> List[int]:
    """Retrieve calories carried by each elf"""
    content = Path(path).read_text(encoding="utf-8")
    return [
        sum(int(line) for line in block.strip().split("\n"))
        for block in content.split("\n\n")
    ]

calories = pd.Series(get_calories(input_path), name="calories")
calories


0      55684
1      43399
2      48846
3      48058
4      52061
       ...  
245    22318
246    48674
247    43489
248    47659
249    50009
Name: calories, Length: 250, dtype: int64

In [111]:
# Get the top calories
calories.max()


69693

In [12]:
# Get the sum of calories from top 3
calories.sort_values(ascending=False).head(3).sum()


200945

## With list instead of pandas

In [10]:
def get_calories(path: Path) -> List[int]:
    """Retrieve calories carried by each elf"""
    content = Path(path).read_text(encoding="utf-8")
    return [
        sum(int(line) for line in block.strip().splitlines())
        for block in content.split("\n\n")
    ]

calories = get_calories(input_path)
max(calories), sum(sorted(calories, reverse=True)[:3])


(69693, 200945)

# 02 December 2022


In [160]:
%get_daily_input 2
data[:100]

'A Z\nA Y\nB X\nB X\nC X\nB X\nA X\nA X\nC X\nA X\nA X\nA Y\nB X\nA Y\nC X\nC X\nA X\nA Y\nC X\nB X\nA X\nB X\nA X\nA X\nB Y\n'

In [176]:
import enum


class Gesture(enum.IntEnum):
    ROCK = 0
    PAPER = 1
    SCISSORS = 2

GESTURES = list(Gesture)

In [177]:
from typing import Literal

ABC = Literal["A", "B", "C"]
XYZ = Literal["X", "Y", "Z"]


def get_points(other_letter: ABC, my_letter: XYZ) -> int:
    """Get the points for a given combination of letters"""
    # We will use the values of the enum
    other = GESTURES["ABC".index(other_letter)]
    me = GESTURES["XYZ".index(my_letter)]

    points_choice = me.value + 1
    if other == me:
        return 3 + points_choice

    if me.value == (other.value + 1) % 3:
        return 6 + points_choice

    return points_choice


get_points("A", "Y"), get_points("B", "X"), get_points("C", "Z")


(8, 1, 6)

In [178]:
mini_data = """
A Y
B X
C Z
"""

In [179]:
import re


def get_points_from_list(data: str, fn=get_points) -> int:
    """Get the points from the data"""
    return sum(
        fn(other, my) for other, my in re.findall(r"([ABC]) ([XYZ])", data)
    )


(get_points_from_list(mini_data), get_points_from_list(data))


(15, 11150)

In [181]:
def get_points_2(other_letter: ABC, my_letter: XYZ) -> int:
    """Get the points for a given combination of letters (2nd scenario)"""
    other = GESTURES["ABC".index(other_letter)]

    # X means you loose, Y means you draw, Z means you win
    offset = "XYZ".index(my_letter) - 1
    me = GESTURES[(other.value + offset) % 3]

    points_choice = me.value + 1
    if other == me:
        return 3 + points_choice

    if me.value == (other.value + 1) % 3:
        return 6 + points_choice

    return points_choice


assert get_points_from_list(mini_data, fn=get_points_2) == 6 + 3 + 3
get_points_from_list(data, fn=get_points_2)

8295

# Day 3

In [139]:
%get_daily_input 3
data[:100]

'BdbzzddChsWrRFbzBrszbhWMLNJHLLLLHZtSLglFNZHLJH\nnnfMwqpQTMffHlNNLllHnZSS\ncGpcMwfppfqcjcTCBBzWDsDbDrjz'

In [142]:
MINI_DATA = """\
vJrwpWtwJgWrhcsFMMfFFhFp
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
ttgJtRGJQctTZtZT
CrZsJsPPZsGzwwsLwLmpwMDw
"""

In [150]:
from string import ascii_lowercase, ascii_uppercase

def get_line_priority(line: str) -> int:
    """Get the priority of each rucksack line"""
    half_length = len(line) // 2
    left, right = line[:half_length], line[half_length:]
    common_letter = set(left).intersection(set(right)).pop()
    # Find the priority of the common letter
    letters = ascii_lowercase + ascii_uppercase
    return letters.index(common_letter) + 1

get_line_priority("vJrwpWtwJgWrhcsFMMfFFhFp"), get_line_priority("jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL")

(16, 38)

In [151]:
sum(get_line_priority(line) for line in data.splitlines())

7889

In [148]:
from typing import Generator


def get_badge_priority(txt: str) -> Generator[int, None, None]:
    """Get the priorities of the badges, by group of 3"""
    lines = txt.splitlines()
    for i in range(0, len(lines), 3):
        common_letters = (
            set(lines[i])
            .intersection(set(lines[i + 1]))
            .intersection(set(lines[i + 2]))
        )
        letters = ascii_lowercase + ascii_uppercase
        yield min(letters.index(letter) + 1 for letter in common_letters)


sum(get_badge_priority(MINI_DATA))


70

In [149]:
sum(get_badge_priority(data))

2825

# Day 4

In [128]:
%get_daily_input 4
data[:100]

'91-93,6-92\n85-97,18-63\n15-99,16-98\n35-49,34-50\n4-7,6-93\n57-86,57-58\n1-82,83-90\n21-88,20-89\n83-84,1-8'

In [129]:
MINI_DATA = """\
2-4,6-8
2-3,4-5
5-7,7-9
2-8,3-7
6-6,4-6
2-6,4-8"""

In [131]:
def is_fully_contained(line:str) -> bool:
    """Check if the line is fully contained in another line"""
    left, right = line.split(",")
    left_min, left_max = [int(e) for e in left.split("-")]
    right_min, right_max = [int(e) for e in right.split("-")]
    return (left_min <= right_min and left_max >= right_max) or (right_min <= left_min and right_max >= left_max)

is_fully_contained("6-6,4-6"), is_fully_contained("2-6,4-8")

(True, False)

In [132]:
sum(is_fully_contained(line) for line in MINI_DATA.splitlines())

2

In [133]:
sum(is_fully_contained(line) for line in data.splitlines())

456

In [135]:
def is_overlap(line:str) -> bool:
    """Check if the line contains overlap"""
    left, right = line.split(",")
    left_min, left_max = [int(e) for e in left.split("-")]
    right_min, right_max = [int(e) for e in right.split("-")]
    left_range = set(range(left_min, left_max + 1))
    right_range = set(range(right_min, right_max + 1))
    return bool(left_range.intersection(right_range))

is_overlap("6-6,4-6"), is_overlap("2-6,4-8"), is_overlap("4-8,2-6"), is_overlap("2-4,6-8")

(True, True, True, False)

In [136]:
sum(is_overlap(line) for line in data.splitlines())

808

# Day 5

In [213]:
%get_daily_input 5

In [183]:
CRATES = """\
            [M] [S] [S]
        [M] [N] [L] [T] [Q]
[G]     [P] [C] [F] [G] [T]
[B]     [J] [D] [P] [V] [F] [F]
[D]     [D] [G] [C] [Z] [H] [B] [G]
[C] [G] [Q] [L] [N] [D] [M] [D] [Q]
[P] [V] [S] [S] [B] [B] [Z] [M] [C]
[R] [H] [N] [P] [J] [Q] [B] [C] [F]
"""

In [224]:
MINI_DATA = """\
    [D]
[N] [C]
[Z] [M] [P]
 1   2   3

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
"""

In [186]:
import re
line = CRATES.splitlines()[0]

line

'            [M] [S] [S]'

In [219]:
from collections import defaultdict
from typing import Dict, List

# List of crates from the top
def get_crates(data: str) -> Dict[int, List[str]]:
    """Get list of crates as dictionary"""
    all_crates = defaultdict(list)
    for line in data.splitlines():
        crates = (c.strip("[ ]") for c in re.findall(r"(\[[A-Z]\]| {3})(?: |$)", line, flags=re.M))
        for i, crate in enumerate(crates, start=1):
            if crate:
                all_crates[i].append(crate)
    return all_crates

all_crates = get_crates(data)
all_crates


defaultdict(list,
            {4: ['M', 'N', 'C', 'D', 'G', 'L', 'S', 'P'],
             5: ['S', 'L', 'F', 'P', 'C', 'N', 'B', 'J'],
             6: ['S', 'T', 'G', 'V', 'Z', 'D', 'B', 'Q'],
             3: ['M', 'P', 'J', 'D', 'Q', 'S', 'N'],
             7: ['Q', 'T', 'F', 'H', 'M', 'Z', 'B'],
             1: ['G', 'B', 'D', 'C', 'P', 'R'],
             8: ['F', 'B', 'D', 'M', 'C'],
             9: ['G', 'Q', 'C', 'F'],
             2: ['G', 'V', 'H']})

In [199]:
commands = 
commands

[('1', '2', '1'), ('3', '1', '3'), ('2', '2', '1'), ('1', '1', '2')]

In [222]:
values = [1, 2, 3, 4, 5]
# Get first 3 values with pop
values.insert(0, 10)
values

[10, 1, 2, 3, 4, 5]

In [230]:
def part_1(pb_input:str):
    all_crates = get_crates(pb_input)

    commands = re.findall(r"move (\d+) from (\d) to (\d)", pb_input)
    for qty, from_pile, to_pile in commands:
        from_pile = int(from_pile)
        to_pile = int(to_pile)
        for _ in range(int(qty)):
            crate = all_crates[from_pile].pop(0)
            all_crates[to_pile].insert(0, crate)

    solution = ''.join(c[0] for nb, c in sorted(all_crates.items()))
    print(f"Solution: {solution}")
    return all_crates

part_1(MINI_DATA)

Solution: CMZ


defaultdict(list, {2: ['M'], 1: ['C'], 3: ['Z', 'N', 'D', 'P']})

In [231]:
part_1(data)

Solution: TLNGFGMFN


defaultdict(list,
            {4: ['G',
              'L',
              'Q',
              'H',
              'H',
              'V',
              'M',
              'C',
              'F',
              'D',
              'S',
              'J',
              'J',
              'Q',
              'G',
              'D',
              'S'],
             5: ['F', 'S'],
             6: ['G'],
             3: ['N'],
             7: ['M'],
             1: ['T',
              'C',
              'N',
              'Z',
              'B',
              'B',
              'B',
              'D',
              'G',
              'T',
              'D',
              'C',
              'M',
              'M',
              'R',
              'P',
              'P',
              'B',
              'Z',
              'Q',
              'V',
              'G',
              'C',
              'B',
              'P',
              'Q',
              'F',
              'D',
              'C',
    

In [232]:
def part_2(pb_input:str):
    all_crates = get_crates(pb_input)

    commands = re.findall(r"move (\d+) from (\d) to (\d)", pb_input)
    for qty, from_pile, to_pile in commands:
        qty = int(qty)
        from_pile = int(from_pile)
        to_pile = int(to_pile)

        all_crates[to_pile] = all_crates[from_pile][:qty] + all_crates[to_pile]
        all_crates[from_pile] = all_crates[from_pile][qty:]

    solution = ''.join(c[0] for nb, c in sorted(all_crates.items()))
    print(f"Solution: {solution}")
    return all_crates

part_2(MINI_DATA)

Solution: MCD


defaultdict(list, {2: ['C'], 1: ['M'], 3: ['D', 'N', 'Z', 'P']})

In [233]:
part_2(data)

Solution: FGLQJCMBD


defaultdict(list,
            {4: ['Q',
              'T',
              'Q',
              'C',
              'P',
              'B',
              'D',
              'N',
              'T',
              'M',
              'C',
              'V',
              'V',
              'Z',
              'F',
              'H',
              'G'],
             5: ['J', 'B'],
             6: ['C'],
             3: ['L'],
             7: ['M'],
             1: ['F',
              'S',
              'S',
              'G',
              'N',
              'D',
              'M',
              'S',
              'Q',
              'P',
              'H',
              'N',
              'M',
              'C',
              'G',
              'F',
              'B',
              'P',
              'D',
              'G',
              'R',
              'D',
              'Q',
              'S',
              'Z',
              'C',
              'J',
              'F',
              'P',
    

# Day 6

In [238]:
%get_daily_input 6 2022

In [247]:
def get_index_end_marker(signal:str, size:int = 4) -> int:
    """Get the index of the end marker after <size> different characters"""
    for i in range(len(signal) - size):
        if len(set(signal[i:i+size])) == size:
            return i + size
    return 0

mini_data = """\
bvwbjplbgvbhsrlpgdmjqwftvncz
nppdvjthqldpwncqszvftbrmjlhg
nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg
zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw
"""

for line in mini_data.splitlines():
    print(get_index_end_marker(line))

get_index_end_marker(data)

5
6
10
11


1892

In [246]:
get_index_end_marker(data, size=14)

2313

# Day 7

In [ ]:
submit()

In [4]:
%get_daily_input 7 2022



In [5]:
MINI_DATA = """\
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
"""

In [14]:
from collections import defaultdict
from typing import Dict
from pathlib import Path

def get_size_dirs(data:str) -> Dict[str, int]:
    """Get size of directories"""
    size_dirs = defaultdict(int)
    current_dir = Path("/")
    for line in data.splitlines():
        if line.startswith("$ cd .."):
            current_dir = current_dir.parent
        elif line.startswith("$ cd "):
            current_dir = current_dir / line[5:]
        elif line.startswith("$ ls"):
            continue
        elif line.startswith("dir "):
            # dir_name = line[4:]
            # size_dirs[current_dir] += size_dirs[dir_name]
            continue
        else:
            size, _ = line.split()
            size_dirs[current_dir] += int(size)
            for p in current_dir.parents:
                size_dirs[p] += int(size)
    return size_dirs

size_mini_data = get_size_dirs(MINI_DATA)
size_mini_data

defaultdict(int,
            {WindowsPath('/'): 48381165,
             WindowsPath('/a'): 94853,
             WindowsPath('/a/e'): 584,
             WindowsPath('/d'): 24933642})

In [16]:
sum(v for d, v in size_mini_data.items() if v < 100000)

95437

In [17]:
sum(v for v in get_size_dirs(data).values() if v < 100000)

1642503

In [18]:
# submit(1642503, day=7)

answer a: None
submitting for part a


That's the right answer!  You are one gold star closer to collecting enough star fruit. You achieved rank 66 on this star's leaderboard and gained 35 points! [Continue to Part Two]


<Response [200]>

In [39]:
TOTAL_SIZE = 70000000
NEEDED_SPACE = 30000000
[
    min(
        v
        for v in size_mini_data.values()
        if TOTAL_SIZE - size_mini_data[Path("/")] + v > NEEDED_SPACE
    ),
    min(
        v for v in size_mini_data.values() if v > TOTAL_SIZE - size_mini_data[Path("/")]
    ),
]


[24933642, 24933642]

In [21]:
size_data = get_size_dirs(data)
size_data


defaultdict(int,
            {WindowsPath('/'): 46592386,
             WindowsPath('/fts'): 618943,
             WindowsPath('/fts/dlqtffw'): 73533,
             WindowsPath('/fts/rbfmmjvd'): 290697,
             WindowsPath('/jnwr'): 851171,
             WindowsPath('/jnwr/wbv'): 461954,
             WindowsPath('/jnwr/wbv/nmdwbnnr'): 208370,
             WindowsPath('/jnwr/zzbvdcf'): 8052,
             WindowsPath('/lrvl'): 22022022,
             WindowsPath('/lrvl/bqqltcg/dlbjblbf'): 11732,
             WindowsPath('/lrvl/bqqltcg'): 2980003,
             WindowsPath('/lrvl/bqqltcg/fdlw'): 414205,
             WindowsPath('/lrvl/bqqltcg/fdlw/hlvpfw/dhwq'): 185769,
             WindowsPath('/lrvl/bqqltcg/fdlw/hlvpfw'): 185769,
             WindowsPath('/lrvl/bqqltcg/jnwr'): 521337,
             WindowsPath('/lrvl/bqqltcg/slzdbm'): 161906,
             WindowsPath('/lrvl/bqqltcg/zcgrgff'): 1870823,
             WindowsPath('/lrvl/bqqltcg/zcgrgff/ztrslh'): 1584583,
             WindowsP

In [35]:
solution = min(v for v in size_data.values() if TOTAL_SIZE - size_data[Path("/")] + v > NEEDED_SPACE)
print(f"Solution: {solution}")

Solution: 6999588


# Day 8

In [5]:
%get_daily_input 8

In [6]:
MINI_DATA = """\
30373
25512
65332
33549
35390
"""

In [57]:
# Check trees visible from outside based on height

import numpy as np
import numpy.typing as npt

IntMatrix = npt.NDArray[np.int64]
BoolMatrix = npt.NDArray[np.bool_]


def get_trees(data: str) -> IntMatrix:
    matrix = np.array([[int(nb) for nb in line] for line in data.splitlines()])
    return matrix


matrix = get_trees(MINI_DATA)
matrix


array([[3, 0, 3, 7, 3],
       [2, 5, 5, 1, 2],
       [6, 5, 3, 3, 2],
       [3, 3, 5, 4, 9],
       [3, 5, 3, 9, 0]])

In [26]:
# True if all nb before is bigger OR all nb after is bigger or all number above are bigger or all number below are bigger
def is_visible(matrix: IntMatrix) -> BoolMatrix:
    """Create a boolean matrix where True means the number is visible"""
    visible = np.zeros_like(matrix, dtype=bool)
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if (
                np.all(matrix[i, :j] < matrix[i, j])
                or np.all(matrix[i, j + 1 :] < matrix[i, j])
                or np.all(matrix[:i, j] < matrix[i, j])
                or np.all(matrix[i + 1 :, j] < matrix[i, j])
            ):
                visible[i, j] = True
    return visible


visible = is_visible(matrix)
visible


array([[ True,  True,  True,  True,  True],
       [ True,  True,  True, False,  True],
       [ True,  True, False,  True,  True],
       [ True, False,  True, False,  True],
       [ True,  True,  True,  True,  True]])

In [58]:
def compute_nb_visible(data:str) -> int:
    """Compute the number of visible trees"""
    matrix = get_trees(data)
    visible = is_visible(matrix)
    return np.sum(visible)

compute_nb_visible(MINI_DATA)

21

In [21]:
solution = compute_nb_visible(data)
print(f"Solution: {solution}")
# submit(solution, day=8)

1851

In [53]:
from typing import List

IntArray = npt.NDArray[np.int64]

def get_nb_trees_visible(height: int, other_trees: IntArray) -> int:
    """Compute the number of trees visible from the current tree"""
    # print(height, other_trees)
    nb_trees_visible = 0
    for other_height in other_trees:
        nb_trees_visible += 1
        if other_height >= height:
            break

    return nb_trees_visible


def get_viewing_distance(matrix:IntMatrix) -> IntMatrix:
    """Create an integer matrix where the value is the multiplication of all numbers visible from the current number"""
    score = np.zeros_like(matrix, dtype=int)
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            height = matrix[i, j]
            # print(f"Position: {i, j}, height: {height}")
            score[i, j] = (
                get_nb_trees_visible(height, matrix[i, :j][::-1])
                * get_nb_trees_visible(height, matrix[i, j + 1 :])
                * get_nb_trees_visible(height, matrix[:i, j][::-1])
                * get_nb_trees_visible(height, matrix[i + 1 :, j])
            )
    return score

# get_nb_trees_visible(5, [3, 5, 3, 1])

def get_max_score(data:str) -> int:
    """Compute the maximum score for visibility"""
    matrix = get_trees(data)
    scores = get_viewing_distance(matrix)
    return scores.max()

get_max_score(MINI_DATA)

8

In [54]:
solution = get_max_score(data)
print(f"Solution: {solution}")
submit(solution, day=8)


Solution: 574080


answer a: 1851
submitting for part b (part a is already completed)


That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 8! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


<Response [200]>